In [18]:
import pandas as pd
import re
import string

#change dataset path
df = pd.read_csv('data/processed/en_tweets_processed.csv')
df = df[df.language == 'en']
df = df[~df['text'].isna()]
preprocessed_documents = df['text'].tolist()


## Topic Modeling per month (not all data used)

In [19]:
from src.nnmf_pipeline import NNMFPipelineEnglish
from src.lsa_pipeline import LSAPipelineEnglish

In [25]:
NNMFPipeline =  NNMFPipelineEnglish(preprocessed_documents,ngram_range=(1,2),n_components=5,max_iter=1000,tf_idf_max_df=0.9,tf_idf_min_df=4,random_state=0)
topics1 = NNMFPipeline.run_topics_detection()
NNMFPipeline.calculate_coherence_score()

0.7048616049254075

In [22]:
lsa_pipeline = LSAPipelineEnglish(preprocessed_documents,ngram_range=(1,2),lsa_components=5,svd_n_iter=200,tf_idf_max_df=0.9,tf_idf_min_df=4,random_state=0)
topics2 = lsa_pipeline.run_topics_detection()
lsa_pipeline.calculate_coherence_score()

0.7562434732130454

In [26]:
topics1

[['nuclear',
  'plant',
  'power',
  'power plant',
  'nuclear power',
  'nuclear plant',
  'zaporizhzhia',
  'russia',
  'zaporizhzhia nuclear',
  'disaster'],
 ['ukraine',
  'russia',
  'support',
  'support ukraine',
  'glory',
  'help',
  'aid',
  'glory ukraine',
  'war ukraine',
  'people ukraine'],
 ['russian',
  'ukrainian',
  'video',
  'force',
  'military',
  'air',
  'defense',
  'soldier',
  'region',
  'air defense'],
 ['passive',
  'awesome',
  'hire',
  'term',
  'website',
  'income',
  'long term',
  'term passive',
  'create',
  'autopilot'],
 ['war',
  'russia',
  'like',
  'people',
  'putin',
  'world',
  'country',
  'need',
  'day',
  'time']]

In [24]:
topics2

[['passive',
  'awesome',
  'hire',
  'term',
  'website',
  'income',
  'long term',
  'term passive',
  'create',
  'autopilot'],
 ['ukraine',
  'russian',
  'russia',
  'war',
  'nuclear',
  'plant',
  'ukrainian',
  'putin',
  'people',
  'power'],
 ['nuclear',
  'plant',
  'power',
  'power plant',
  'nuclear power',
  'nuclear plant',
  'zaporizhzhia',
  'zaporizhzhia nuclear',
  'russia',
  'disaster'],
 ['russian',
  'video',
  'ukrainian',
  'force',
  'air',
  'defense',
  'military',
  'air defense',
  'soldier',
  'region'],
 ['ukraine',
  'russian',
  'force',
  'armed force',
  'armed',
  'ukrainian',
  'force ukraine',
  'military',
  'glory',
  'air']]

In [ ]:
pd.read_csv('data/processed/en_tweets_processed.csv')

In [ ]:
0.7048616049254076